In [ ]:
import numpy as np
import pandas as pd
from ast import literal_eval
import gurobipy as gp
from gurobipy import quicksum
from gurobipy import GRB
import csv
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [ ]:
# df = pd.read_csv('onLastModel.csv',converters={'TRPMILES': pd.eval})
df = pd.read_csv('datasets/onLastModelweUse.csv')

In [ ]:
# Convert entire column to a list
df.loc[:,'TRPMILES'] = df.loc[:,'TRPMILES'].apply(lambda x: literal_eval(x))
df.loc[:,'DWELTIME'] = df.loc[:,'DWELTIME'].apply(lambda x: literal_eval(x))
df.loc[:,'states'] = df.loc[:,'states'].apply(lambda x: literal_eval(x))
df.loc[:,'WHYFROM_GENERAL'] = df.loc[:,'WHYFROM_GENERAL'].apply(lambda x: literal_eval(x))
# df.loc['power'] = df.loc[:,'power'].apply(lambda x: literal_eval(x))
# df.loc['ePrice'] = df.loc[:,'ePrice'].apply(lambda x: literal_eval(x))

In [ ]:
def to_float(x):
    for i in range(len(x)):
        x[i] = float(x[i])
    return x

In [ ]:
df.loc[:,"DWELTIME"] = df.loc[:,"DWELTIME"].agg(to_float)
df.loc[:,"TRPMILES"] = df.loc[:,"TRPMILES"].agg(to_float)
# df.loc[:,"power"] = df.loc[:,"power"].agg(to_float)
# df.loc[:,"ePrice"] = df.loc[:,"ePrice"].agg(to_float)

In [ ]:
df.loc[0,'power'][0]

In [ ]:
def powerInput(x):
    powerList = []
    for i in range(len(x)):
        if x[i] == 'Home':
            powerList.append(P[0])
        elif x[i] == 'Work':
            powerList.append(P[1])
        else:
            powerList.append(P[2])
    return powerList

def electricityPriceInput(x1,x2):
    ePriceList = []
    for i in range(len(x2)):
        if x2[i] == 'Home':
            ePriceList.append(e_level12.iloc[x1-1,2])
        elif x2[i] == 'Work':
            ePriceList.append(e_workplace)
        else:
            ePriceList.append(e_level3)
            
    return ePriceList

In [ ]:
df['power'] = df["WHYFROM_GENERAL"].agg(powerInput)

In [ ]:
df['ePrice']=df.apply(lambda x: electricityPriceInput(x['CENSUS_D'],x['WHYFROM_GENERAL']), axis=1)

parameters

In [ ]:
# full range miles of BEV
r_full = 100
# Energy efficiency of the vehicle (miles/kWh)
eta = 3.3
# Charging power rate three levels (kW)
P = [6, 6, 50]
# electricity price of level 3 (unit: $/kWh)
e_level3 = 0.35
# electrcity price of level 2 at workplace (unit: $/kWh)
e_workplace = 0.33
# electricity price of two levels in different regions ($/kWh)
e_level12 = pd.read_excel('datasets/parameter electricity fee and charging power.xlsx', 
                  sheet_name='electricityPriceLevel12 ')
#intial remaining miles
r = 30

# value= cr_ij/Pcr_ij*e_ij

In [ ]:
# add two columns illustrate the electricity price and charger power of different location

In [ ]:
aaa = sample.head(1)
# aaa['states']

In [ ]:
# chargeAvil(aaa)
num=len(aaa['states'])
rLst = np.zeros((num-1))

In [ ]:
sample['DWELTIME'][1][2]

In [ ]:
test = sample.iloc[1,:]
test['states']

In [ ]:
# Notice this model is just for one household vehicle 

def chargeAvil(test):
    num=len(test['states'])
    # Dwell time of MUDs household i at location j
    DT = np.zeros((num-1))
#     rLst = np.zeros((num-1))
    for j in range(num-1):
        DT[j] = test['DWELTIME'][j]

    # distance of MUDs household i from location j to the next location
    d = np.zeros(num-1)
    for j in range(num-1):
        d[j] = test['TRPMILES'][j]

    # Create a new model
    m = gp.Model()
    m.Params.LogToConsole = 0
    #     m = gp.Model("bilinear")
    # m.params.NonConvex = 2

    # Create variables x_ni and y_ni
    # charging availability at MUDs
    #mud_or = [i for i in range(I)]
    #mud_im = [j in range(num - 1)]
    x = m.addVar(vtype=GRB.BINARY, name="charging Decision at MUDs")
    # charging time of MUDs household i at location j
    y = m.addVars(num-1,lb=0.0, ub=float('inf'), vtype=GRB.CONTINUOUS, name="charge time")

    # # Set objective: 
    obj = quicksum(test['ePrice'][j]*test['power'][j]*y[j] for j in range(num - 1))
    m.setObjective(obj,GRB.MINIMIZE)

    for j in range(num - 1):
        if test['WHYFROM_GENERAL'][j] == 'Home':
            m.addConstr((1 - x)*y[j] == 0,'bilinear')
        if (test['WHYFROM_GENERAL'][j] == 'Public')& (test['public_charge_assign'] == '0'):
            m.addConstr(y[j] == 0,'bilinear')
        if (test['WHYFROM_GENERAL'][j] == 'Work')& (test['work_charge_assign'] == '0'):
            m.addConstr(y[j] == 0,'bilinear')

        for j in range(num - 2):
    #         m.addConstr((r[i,j+1] == r[i,j] - d[i,j] + 12.5*y[i,j]),'state transition function')
    #         m.addConstr((r[i,j+1] >= 0.2*r_full),'Battery storage capacity')
            m.addConstr((r - quicksum(d[k] for k in range(j)) 
                         + eta*test['power'][j]*quicksum(y[k] for k in range(j)) >= 0.2*r_full),'state transition function')
        for j in range(num - 1):
            m.addConstr((y[j] <= DT[j]/60),'charging time less than dwell time')
            m.addConstr((r - quicksum(d[k] for k in range(j)) 
                         + eta*test['power'][j]*quicksum(y[k] for k in range(j)) <= r_full),'charging electricity less battery capacity')

        m.addConstr((r - quicksum(d[k] for k in range(num-1)) 
                    + eta*test['power'][j]*quicksum(y[k] for k in range(num-1)) >= 20),'range miles end')
    #     m.addConstr((r - quicksum(d[k] for k in range(num-1)) 
    #                  + eta*test['power'][j]*quicksum(y[k] for k in range(num-1)) >= 0.8*r_full),'range miles end')

        # First optimize() call will fail - need to set NonConvex to 2


    # rLst[0] = r
    # for j in range(num - 2):
    #     m.addConstr((rLst[j+1] == rLst[j] - d[j] + eta*test['power'][j]*y[j]),'state transition function')

    # for j in range(num - 1):
    #     m.addConstr((0.2*r_full <= rLst[j+1]),'Battery storage capacity safety')
    #     m.addConstr((y[j] <= DT[j]/60),'charging time less than dwell time')
    # m.addConstr((0.2*r_full <= rLst[num - 1]),'last stop battery')
    # First optimize() call will fail - need to set NonConvex to 2
    try:
        m.optimize()
    except gp.GurobiError:
        print("Optimize failed due to non-convexity")
    sol=[]
    if m.status ==GRB.OPTIMAL:
        for v in m.getVars():
            sol.append(v.x)
#             print('%s %g' % (v.varName, v.x))
    else: sol='infeasible'
        
    return sol

In [ ]:
results = sample.agg(chargeAvil,axis=1)

In [ ]:
df['power'][0][0]

In [ ]:
df['power'] = df["WHYFROM_GENERAL"].agg(powerInput)
df['ePrice']=df.apply(lambda x: electricityPriceInput(x['CENSUS_D'],x['WHYFROM_GENERAL']), axis=1)

In [ ]:
df.head()

In [ ]:
resultsAll = df.agg(chargeAvil,axis=1)

In [ ]:
df['resultsS3E2'] = resultsAll

In [ ]:
r = 50
r

In [ ]:
# df['resultsS5E2'] = resultsAllS5E2
df.head(20)

In [ ]:
resultsAllS5E8 = df.agg(chargeAvilE8,axis=1)
df['resultsS5E8'] = resultsAllS5E8
df.head(20)

In [ ]:
resultsAllS5E2 = df.agg(chargeAvil,axis=1)

In [ ]:
r = 30
# resultsAllS3E2 = df.agg(chargeAvil,axis=1)
# df['resultsS3E2'] = resultsAllS3E2
resultsAllS3E8 = df.agg(chargeAvilE8,axis=1)
df['resultsS3E8'] = resultsAllS3E8

In [ ]:
df.head()

In [ ]:
# Notice this model is just for one household vehicle 

def chargeAvilE8(test):
    num=len(test['states'])
    # Dwell time of MUDs household i at location j
    DT = np.zeros((num-1))
#     rLst = np.zeros((num-1))
    for j in range(num-1):
        DT[j] = test['DWELTIME'][j]

    # distance of MUDs household i from location j to the next location
    d = np.zeros(num-1)
    for j in range(num-1):
        d[j] = test['TRPMILES'][j]

    # Create a new model
    m = gp.Model()
    m.Params.LogToConsole = 0
    #     m = gp.Model("bilinear")
    # m.params.NonConvex = 2

    # Create variables x_ni and y_ni
    # charging availability at MUDs
    #mud_or = [i for i in range(I)]
    #mud_im = [j in range(num - 1)]
    x = m.addVar(vtype=GRB.BINARY, name="charging Decision at MUDs")
    # charging time of MUDs household i at location j
    y = m.addVars(num-1,lb=0.0, ub=float('inf'), vtype=GRB.CONTINUOUS, name="charge time")

    # # Set objective: 
    obj = quicksum(test['ePrice'][j]*test['power'][j]*y[j] for j in range(num - 1))
    m.setObjective(obj,GRB.MINIMIZE)

    for j in range(num - 1):
        if test['WHYFROM_GENERAL'][j] == 'Home':
            m.addConstr((1 - x)*y[j] == 0,'bilinear')
        if (test['WHYFROM_GENERAL'][j] == 'Public')& (test['public_charge_assign'] == '0'):
            m.addConstr(y[j] == 0,'bilinear')
        if (test['WHYFROM_GENERAL'][j] == 'Work')& (test['work_charge_assign'] == '0'):
            m.addConstr(y[j] == 0,'bilinear')

        for j in range(num - 2):
    #         m.addConstr((r[i,j+1] == r[i,j] - d[i,j] + 12.5*y[i,j]),'state transition function')
    #         m.addConstr((r[i,j+1] >= 0.2*r_full),'Battery storage capacity')
            m.addConstr((r - quicksum(d[k] for k in range(j)) 
                         + eta*test['power'][j]*quicksum(y[k] for k in range(j)) >= 0.2*r_full),'state transition function')
        for j in range(num - 1):
            m.addConstr((y[j] <= DT[j]/60),'charging time less than dwell time')
            m.addConstr((r - quicksum(d[k] for k in range(j)) 
                         + eta*test['power'][j]*quicksum(y[k] for k in range(j)) <= r_full),'charging electricity less battery capacity')

        m.addConstr((r - quicksum(d[k] for k in range(num-1)) 
                    + eta*test['power'][j]*quicksum(y[k] for k in range(num-1)) >= 80),'range miles end')
    #     m.addConstr((r - quicksum(d[k] for k in range(num-1)) 
    #                  + eta*test['power'][j]*quicksum(y[k] for k in range(num-1)) >= 0.8*r_full),'range miles end')

        # First optimize() call will fail - need to set NonConvex to 2


    # rLst[0] = r
    # for j in range(num - 2):
    #     m.addConstr((rLst[j+1] == rLst[j] - d[j] + eta*test['power'][j]*y[j]),'state transition function')

    # for j in range(num - 1):
    #     m.addConstr((0.2*r_full <= rLst[j+1]),'Battery storage capacity safety')
    #     m.addConstr((y[j] <= DT[j]/60),'charging time less than dwell time')
    # m.addConstr((0.2*r_full <= rLst[num - 1]),'last stop battery')
    # First optimize() call will fail - need to set NonConvex to 2
    try:
        m.optimize()
    except gp.GurobiError:
        print("Optimize failed due to non-convexity")
    sol=[]
    if m.status ==GRB.OPTIMAL:
        for v in m.getVars():
            sol.append(v.x)
#             print('%s %g' % (v.varName, v.x))
    else: sol='infeasible'
        
    return sol

In [ ]:
resultsAllS8E8 = df.agg(chargeAvilS8E8,axis=1)

In [ ]:
df['resultsS8E2'] = resultsAllS8E2
df['resultsS8E8'] = resultsAllS8E8


In [ ]:
# r=50

# resultsAllS5E2 = df.agg(chargeAvil,axis=1)
# resultsAllS5E8 = df.agg(chargeAvilE8,axis=1)

df['resultsS5E2'] = resultsAllS5E2
df['resultsS5E8'] = resultsAllS5E8

# df.to_csv('onLastModelweUseW033P035.csv')

In [ ]:
r

In [ ]:
resultsAllS5E2

In [ ]:
df.to_csv('onLastModelweUseW033P035.csv')

In [ ]:
df['resultsS5E2'] = resultsAllS5E2
df['resultsS5E8'] = resultsAllS5E8

In [ ]:
df.tail()

In [ ]:
# r = 20
# r
# resultsAllS2E8 = df.agg(chargeAvilS8E8,axis=1)

df['resultsS2E8'] = resultsAllS2E8
print('DONE!!')
# df.to_csv('onLastModelweUseR1.csv')
# df.head()

In [ ]:
df.to_csv('onLastModelweUseR1.csv')

In [ ]:
df.head()

In [ ]:
resultsAllS2E2

In [ ]:
def check_end80(x):
    if type(x)==list:
        if x[-1] > 0:
            return 1
        return x[0]
    else:
        return 0